In [0]:
from pyspark.sql.functions import (
    col, count, avg, sum,
    to_date, hour
)

#Vamos a leer los datos que ya tenemos en silver
df_silver = spark.read.table("workspace.silver.hvfhs")
print(f"Registros en Silver: {df_silver.count()}")


#De acuerdo al enunciado se requiere tener claridad en la fecha y la hora por lo que creamos las respectivas columnas
df_time = (
    df_silver
    .withColumn("trip_date", to_date("pickup_datetime"))
    .withColumn("trip_hour", hour("pickup_datetime"))
)



In [0]:
#Ahora vamos a calcular los viajes promedio por hora
df_trips_per_hour = (
    df_time
    .groupBy("trip_date", "trip_hour")
    .agg(count("*").alias("trips"))
)

df_avg_trips_per_hour = (
    df_trips_per_hour
    .groupBy("trip_date")
    .agg(avg("trips").alias("avg_trips_per_hour"))
)

#Vamos a ver detalladamente los viajes por ora para una fecha especifica, asi corroboramos la veracidad del resultado
df_trips_per_hour \
    .filter(col("trip_date") == "2025-01-01") \
    .orderBy("trip_hour") \
    .show(24)

#imprimimos los viajes en los primeros dias de enero par ver si coincide con el dato anterior (el promedio se puede sacar manualmente para verificar)
df_avg_trips_per_hour.orderBy("trip_date").show(10, truncate=False)




In [0]:
#De acuerdo a las tarifas que se manejan en NY, ahora vamos a ahhar los ingresos totales
df_revenue = (
    df_time
    .withColumn(
        "total_revenue",
        col("base_passenger_fare") +
        col("tolls") +
        col("bcf") +
        col("sales_tax") +
        col("congestion_surcharge") +
        col("airport_fee") +
        col("cbd_congestion_fee")
    )
    .groupBy("trip_date")
    .agg(sum("total_revenue").alias("total_revenue"))
)

#Imprimimos los primeros resultados para verificar la operacion
df_revenue.orderBy("trip_date").show(10, truncate=False)


In [0]:
#Calculamos ahora el tiempo promedio de viaje
df_avg_duration = (
    df_time
    .groupBy("trip_date")
    .agg(avg("trip_duration_minutes").alias("avg_trip_duration_minutes"))
)

# imprimimos los primeros registros
df_avg_duration.orderBy("trip_date").show(10, truncate=False)


In [0]:
#Calculamos la distancia promedio de viaje
df_avg_distance = (
    df_time
    .groupBy("trip_date")
    .agg(avg("trip_miles").alias("avg_trip_distance_miles"))
)

#imprimimos los primeros registros
df_avg_distance.orderBy("trip_date").show(10,truncate=False)

In [0]:
#Una vez ya tenemos los calculos hechos vamos a unir los KPIs en un arreglo (vamos a juntar los dataFrames resultantes)

from functools import reduce

dfs = [
    df_avg_trips_per_hour,
    df_revenue,
    df_avg_duration,
    df_avg_distance
]

df_gold = reduce(
    lambda left, right: left.join(right, on="trip_date", how="inner"),
    dfs
)

#Imprimimos los primeros registros para verificar antes de guardar
df_gold.orderBy("trip_date").show(10, truncate=False)


In [0]:
#Una vez ya tenemos todos los Kpis procedemos a guardarlo en la capa Gold

(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.gold.hvfhs_kpis_diarios")
)

print("✔️ KPIs diarios creados en la capa Gold")


In [0]:
#Verificamos que los datos hayan persistido correctamente con una consulta rapida

spark.table("workspace.gold.hvfhs_kpis_diarios") \
     .orderBy("trip_date") \
     .show(10, truncate=False)

#Contamos los registros totales
df_total = spark.table("workspace.gold.hvfhs_kpis_diarios").count()
print(f"Total de registros: {df_total}")

